<h1 align ="center"> REST API Video Samples</h1>
<hr>
   
# Chat Completions

In [ ]:
import json
import os
import requests
import time

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup vision_api_key, vision_api_endpoint, deployment_name, openai_api_base, openai_api_key and openai_api_version.

In [ ]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the vision resource key
vision_api_key = os.getenv("VISION_API_KEY")

# The base URL for your vision resource endpoint, e.g. "https://<your-resource-name>.cognitiveservices.azure.com"
# You must create your resource in the East US region.
vision_api_endpoint = config_details['VISION_API_ENDPOINT'] 

# Setting up the deployment name
deployment_name = config_details['GPT-4V_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

### Create Video Index


In [ ]:
video_SAS_url = "your_SAS_url_here" # Insert your video SAS URL, e.g. https://<your-storage-account-name>.blob.core.windows.net/<your-container-name>/<your-video-name>?<SAS-token>
video_index_name = "unique_video_index_name" # This index name must be unique
video_id = "unique_video_id" # This video ID must be unique

def create_video_index(vision_api_endpoint, vision_api_key, index_name):
    url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
    headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
    data = {
        "features": [
            {"name": "vision", "domain": "surveillance"},
            {"name": "speech"}
        ]
    }
    response = requests.put(url, headers=headers, data=json.dumps(data))
    return response

def add_video_to_index(vision_api_endpoint, vision_api_key, index_name, video_url, video_id):
    url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions/my-ingestion?api-version=2023-05-01-preview"
    headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
    data = {
        'videos': [{'mode': 'add', 'documentId': video_id, 'documentUrl': video_url}]
    }
    response = requests.put(url, headers=headers, data=json.dumps(data))
    return response

def wait_for_ingestion_completion(vision_api_endpoint, vision_api_key, index_name, max_retries=30):
    url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions?api-version=2023-05-01-preview"
    headers = {"Ocp-Apim-Subscription-Key": vision_api_key}
    retries = 0
    while retries < max_retries:
        time.sleep(10)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            state_data = response.json()
            if state_data['value'][0]['state'] == 'Completed':
                print(state_data)
                print('Ingestion completed.')
                return True
        retries += 1
    return False


# Step 1: Create an Index
response = create_video_index(vision_api_endpoint, vision_api_key, video_index_name)
print(response.status_code, response.text)

# Step 2: Add a video file to the index
response = add_video_to_index(vision_api_endpoint, vision_api_key, video_index_name, video_SAS_url, video_id)
print(response.status_code, response.text)

# Step 3: Wait for ingestion to complete
if not wait_for_ingestion_completion(vision_api_endpoint, vision_api_key, video_index_name):
    print("Ingestion did not complete within the expected time.")


### Call GPT-4V

In [ ]:
# System messages and user prompt
sys_message = """
Your task is to assist in analyzing and optimizing creative assets. 
You will be presented with advertisement videos for products. 
First describe the video in detail paying close attention to Product characteristics highlighted, 
Background images, Lighting, Color Palette and Human characteristics for persons in the video. 
Finally provide a summary of the video and talk about the main message the advertisement video tries to convey to the viewer. 
"""
user_prompt = "Summarize the ad video"

# Construct the API request URL
api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/extensions/chat/completions?api-version={openai_api_version}"

# Including the api-key in HTTP headers
headers = {
    "Content-Type": "application/json",
    "api-key": openai_api_key,
}

# Payload for the request
payload = {
  "dataSources": [
    {
      "type": "AzureComputerVisionVideoIndex",
      "parameters": {
        "computerVisionResource": f"{vision_api_endpoint}/computervision",
        "computerVisionApiKey": vision_api_key,
        "indexName": video_index_name,
        "videoUrls": [video_SAS_url]
      }
    }
  ],
  "enhancements": {
        "video": {
            "enabled": True
        }
    },
  "messages": [
    {
      "role": "system",
      "content": [
        sys_message
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "acv-document-id": video_id
        },
      ]
    },
    {
      "role": "user",
      "content": [
        user_prompt
      ]
    }, 
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

# Send the request and handle the response
try:
    response = requests.post(api_url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error for bad HTTP status codes
    response_content = response.json()
    print(response_content['choices'][0]['message']['content'])  # Print the content of the response
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")